In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import torch.nn.functional as F
from PIL import Image
print(torch.__version__)
print(torch.cuda.is_available())


1.12.1+cu113
True


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [3]:
train_set = torchvision.datasets.CIFAR10(
    root='./data', train = False, transform = transform)

In [14]:
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size = 10000, shuffle = False, num_workers=0)

In [15]:
test_data_iter = iter(train_loader)
test_image, test_label = test_data_iter.next()

In [16]:
class LeNet(nn.Module): 
    def __init__(self):
        super(LeNet, self).__init__()    
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))   
        x = self.pool1(x)            
        x = F.relu(self.conv2(x))   
        x = self.pool2(x)            
        x = x.view(-1, 32*5*5)     
        x = F.relu(self.fc1(x))     
        x = F.relu(self.fc2(x))     
        x = self.fc3(x)              
        return x


In [17]:
#device = torch.device("cuda")
net = LeNet()
#net.to(device)
loss_function =  nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [24]:
for epoch in range(50):
    running_loss = 0.0
    time_start = time.perf_counter()
    
    for step, data in enumerate(train_loader, start=0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        #print(step)
        if step == 5:
            with torch.no_grad():
                outputs = net(test_image)
                predict_y = torch.max(outputs, dim=1)[1]
                accuray = (predict_y == test_label).sum().item()/test_label.size(0)
                
                print('[%d, %5d] train_loss: %.3f test_accuracy: %.3f' %(epoch+1, step+1, running_loss/500, accuray))
                
                print('%f s' % (time.perf_counter() - time_start))
                running_loss = 0.0
                
print('Finished Training')


save_path = './Lenet.pth'
torch.save(net.state_dict(), save_path)     

[1,     6] train_loss: 0.006 test_accuracy: 0.857
2.681105 s
[2,     6] train_loss: 0.006 test_accuracy: 0.863
2.653401 s
[3,     6] train_loss: 0.005 test_accuracy: 0.866
2.680952 s
[4,     6] train_loss: 0.005 test_accuracy: 0.862
2.685626 s
[5,     6] train_loss: 0.005 test_accuracy: 0.863
2.681384 s
[6,     6] train_loss: 0.005 test_accuracy: 0.865
2.747179 s
[7,     6] train_loss: 0.005 test_accuracy: 0.860
2.674581 s
[8,     6] train_loss: 0.005 test_accuracy: 0.848
2.701328 s
[9,     6] train_loss: 0.005 test_accuracy: 0.846
2.647317 s
[10,     6] train_loss: 0.005 test_accuracy: 0.866
2.664049 s
[11,     6] train_loss: 0.006 test_accuracy: 0.796
2.674323 s
[12,     6] train_loss: 0.006 test_accuracy: 0.867
2.653489 s
[13,     6] train_loss: 0.006 test_accuracy: 0.873
2.667146 s
[14,     6] train_loss: 0.005 test_accuracy: 0.823
2.683831 s
[15,     6] train_loss: 0.006 test_accuracy: 0.790
2.736390 s
[16,     6] train_loss: 0.006 test_accuracy: 0.834
2.693111 s
[17,     6] train

In [25]:
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

im = Image.open('flower.jpg')
im = transform(im)
im = torch.unsqueeze(im, dim=0)

net = LeNet()
net.load_state_dict(torch.load('Lenet.pth'))

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [26]:
with torch.no_grad():
    outputs = net(im)
    predict = torch.max(outputs, dim=1)[1].data.numpy()
    
print(classes[int(predict)])

plane
